## ひろしまQuest2020#stayhome【コース予測部門】

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 300)

### 管理番号

In [2]:
model_No = 20

### sub-modelを使用するかどうか

In [3]:
use_sub_model = True

In [4]:
ALL_MERGE = 'intermediate/all_merge_{}.f'.format(model_No)
SUBMIT = 'submit/course_{}.csv'.format(model_No)

if use_sub_model:
    ALL_MERGE = 'intermediate/all_merge_{}_sub.f'.format(model_No)
    SUBMIT = 'submit/course_{}_sub.csv'.format(model_No)
else:
    OUT_SUBMODEL = 'intermediate/course_predict_{}.f'.format(model_No)

In [5]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 187)

In [6]:
if use_sub_model:
    all_pitch['predict_curve'] = all_pitch['predict_curve'] / all_pitch['predict_straight']
    all_pitch['predict_slider'] = all_pitch['predict_slider'] / all_pitch['predict_straight']
    all_pitch['predict_shoot'] = all_pitch['predict_shoot'] / all_pitch['predict_straight']
    all_pitch['predict_fork'] = all_pitch['predict_fork'] / all_pitch['predict_straight']
    all_pitch['predict_changeup'] = all_pitch['predict_changeup'] / all_pitch['predict_straight']
    all_pitch['predict_sinker'] = all_pitch['predict_sinker'] / all_pitch['predict_straight']
    all_pitch['predict_cutball'] = all_pitch['predict_cutball'] / all_pitch['predict_straight']
    all_pitch.drop(columns=[
        'predict_0', 'predict_1', 'predict_2', 'predict_3', 'predict_4', 'predict_5', 'predict_6',
        'predict_7', 'predict_8', 'predict_9', 'predict_10', 'predict_11', 'predict_12',
        'predict_straight'
    ], inplace=True)

In [7]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 173)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,elapsed_batter,ave_elapsed_game,ave_elapsed_inning,ave_elapsed_batter,diff_elapsed_batter,diff_elapsed_inning,diff_elapsed_game,sayonara,extention,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_count_sum,pre_ball_foul,pre_ball_ball,pre_ball_strike,pre_foul_sum,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_rank_pit,rank_year_pit,rank_x_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,high_str,high_ball,mid_str,low_str,low_ball,left_str,left_ball,center_str,right_str,right_ball,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_rank_bat,rank_year_bat,rank_x_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R,predict_curve,predict_slider,predict_shoot,predict_fork,predict_changeup,predict_sinker,predict_cutball
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.237468,0.325101,0.209547,0.145227,0.092978,0.027829,0.113620,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,0.190270,0,0,0,0,0,1,0,0,0.000000,NaN,0,0.000000,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.468793,0.421637,0.230236,0.041609,0.109570,0.0,0.0,1638.0,27.0,190.0,404.0,0.156899,0.202686,0.214896,0.180098,0.245421,0.186813,0.230769,0.188645,0.176435,0.217338,0,176,74,5.0,15000,0,1,1,0,32,8.0,2.5,0.312500,20.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,0,0,1,0,0.019439,0.053175,0.162372,0.000144,0.001028,0.000008,0.000008
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.210923,0.247198,0.177502,0.320359,0.147952,0.039943,0.116772,0.106283,0.230235,0.157397,0.138195,0.367890,0.184621,0.355720,0.111331,0.105923,0.242405,0,0,0,0,0,1,0,0,0.200000,0.200000,0,0.200000,2,0.200000,0.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0.0,0.0,1.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.468793,0.421637,0.230236,0.041609,0.109570,0.0,0.0,1638.0,27.0,190.0,404.0,0.156899,0.202686,0.214896,0.180098,0.245421,0.186813,0.230769,0.188645,0.176435,0.217338,0,176,74,5.0,15000,0,1,1,0,32,8.0,2.5,0.312500,20.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,0,0,1,0,0.157843,0.229845,0.263541,0.010919,0.039392,0.000025,0.000060
2,2,0.0,5.0,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,1.0,0.138758,0.130040,0.092263,0.419906,0.117327,0.038140,0.046858,0.076007,0.284249,0.102015,0.095055,0.442674,0.118315,0.488095,0.076190,0.078571,0.238828,0,0,0,0,0,1,0,0,0.383333,0.183333,0,0.383333,3,0.383333,0.191667,0.191667,0.191667,-0.008333,-0.008333,-0.008333,0,0,1,0,1

### train
- 行数: 257117

In [8]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 173)

### test
- 行数: 521650

In [9]:
test = all_pitch[all_pitch['course'].isnull()]
test.shape

(521650, 173)

In [10]:
del all_pitch
gc.collect()

40

In [11]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

In [12]:
test_d = test.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

## Dataset作成

In [13]:
lgb_train = lgb.Dataset(train_d, train['course'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [14]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 13,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 8.769293390201968, 
        'lambda_l2': 3.913949617576324e-05, 
        'num_leaves': 6, 
        'feature_fraction': 0.4, 
        'bagging_fraction': 0.8391111798378441, 
        'bagging_freq': 4, 
        'min_child_samples': 50
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 2.32775 + 0.00100935
[200]	cv_agg's multi_logloss: 2.3208 + 0.000969677
[300]	cv_agg's multi_logloss: 2.31828 + 0.000939539
[400]	cv_agg's multi_logloss: 2.31676 + 0.000975521
[500]	cv_agg's multi_logloss: 2.31601 + 0.000856438
[600]	cv_agg's multi_logloss: 2.31571 + 0.000896158
[700]	cv_agg's multi_logloss: 2.31543 + 0.000907502
[800]	cv_agg's multi_logloss: 2.31538 + 0.000925921
Best num_boost_round: 793
Best CV score: 2.315349281506766
CPU times: user 1h 11min 45s, sys: 7min 35s, total: 1h 19min 20s
Wall time: 9min 57s


In [15]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [16]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 21min 56s, sys: 2min 15s, total: 24min 11s
Wall time: 3min 1s


### Feature Importance

In [17]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
119,left_ball,612
125,weight_bat,630
114,high_ball,632
6,pitcher_cnt_in_game,644
137,salary_year_bat,656
122,right_ball,671
45,elapsed_from_inning,694
115,mid_str,715
50,ave_elapsed_batter,749
113,high_str,784


In [18]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 5min 39s, sys: 3.1 s, total: 5min 42s
Wall time: 45.7 s


## 結果出力

In [19]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 14)


,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0.079762,0.132805,0.074496,0.040194,0.053744,0.031691,0.058046,0.050055,0.033072,0.184248,0.093212,0.080730,0.087945
1,1,0.074360,0.137558,0.076428,0.039517,0.048858,0.028300,0.053359,0.056816,0.038825,0.156357,0.103790,0.086075,0.099758
2,2,0.031615,0.086602,0.057300,0.026069,0.037209,0.024249,0.040898,0.054342,0.029299,0.166274,0.105186,0.191816,0.149141
3,3,0.046554,0.140215,0.070892,0.027175,0.035031,0.025483,0.037712,0.050542,0.036936,0.162148,0.092913,0.130153,0.144246
4,4,0.062344,0.138268,0.090329,0.032394,0.037113,0.041110,0.045631,0.067154,0.033093,0.132014,0.060336,0.119945,0.140268


In [20]:
submit.to_csv(SUBMIT, header=False, index=False)

### 球種予測で使用する

In [21]:
%%time
if not use_sub_model:
    train_predict = lgb_model.predict(train_d, num_iteration = lgb_model.best_iteration)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


In [22]:
if not use_sub_model:
    df_train_predict = pd.DataFrame(train_predict).reset_index()
    submodel = pd.concat([df_train_predict, submit], ignore_index=True)
    submodel.drop(columns=['index'], inplace=True)
    submodel.rename(columns={
        0: 'predict_0',
        1: 'predict_1',
        2: 'predict_2',
        3: 'predict_3',
        4: 'predict_4',
        5: 'predict_5',
        6: 'predict_6',
        7: 'predict_7',
        8: 'predict_8',
        9: 'predict_9',
        10: 'predict_10',
        11: 'predict_11',
        12: 'predict_12'
    }, inplace=True)
    print(submodel.shape)
    submodel.head()

In [23]:
if not use_sub_model:
    submodel.to_feather(OUT_SUBMODEL)

### SIGNATE

In [24]:
'signate submit --competition-id=276 ./{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=276 ./submit/course_20_sub.csv --note feat=173_cv=2.315349281506766'

### GCS

In [25]:
'gsutil cp ./{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ./submit/course_20_sub.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 20_sub(173)
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 793
- Best CV score: 2.315349281506766
- Wall time: 9min 57s
- 評価結果  : 2.3585163

#### 20(166) ... player同士の組み合わせを削除
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 572
- Best CV score: 2.3271237073894153
- Wall time: 6min 59s
- 評価結果  : 2.3552697

#### 19_sub(197) 
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 774
- Best CV score: 2.316818574234304
- Wall time: 10min 49s
- 評価結果  : 2.3581029

#### 19(190) 
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 550
- Best CV score: 2.327580481542456
- Wall time: 7min 36s
- 評価結果  : 2.3554862

#### 18_sub(225) ...サブモデルで球種の予測を追加
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 548
- Best CV score: 2.308116414786115
- Wall time: 8min 25s
- 評価結果  : 2.3638683

#### 18(217) 
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 373
- Best CV score: 2.3204478805445055
- Wall time: 6min 9s
- 評価結果  : 2.3604751

#### 17(192) 
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 415
- Best CV score: 2.3200530570975535
- Wall time: 15min 10s
- 評価結果  : 2.3607863

#### 16(181)  ...選手IDごとの2017年のコース実績
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 355
- Best CV score: 2.3203553166835866
- Wall time: 5min 8s
- 評価結果  : 2.3608998

#### 15(153)  ...左右で球種/コースを別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 932
- Best CV score: 2.3387434142884445
- Wall time: 9min 41s
- 評価結果  : 2.3552543 ★

#### 14(153)  ...左右で球種を別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 909
- Best CV score: 2.340082741044867
- Wall time: 9min 40s
- 評価結果  : 2.3555548

#### 3(157)  ...前の投球からの時間差を追加
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 857
- Best CV score: 2.340703597440808
- Wall time: 9min 20s
- 評価結果  : 2.3554087

#### 2(156)  ...merge12のパラメータをチューニング
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 835
- Best CV score: 2.3419206656914
- Wall time: 9min 7s
- 評価結果  : 2.3555299

#### 2(156)  ...merge12(2017のコースデータを付加) 
- 'learning_rate' : 0.1,
- Best num_boost_round: 121
- Best CV score: 2.344104226715559
- Wall time: 12min 30s
- 評価結果  : 2.3564326

#### 1(143)  ...球種予測の11と同じデータ 
- 'learning_rate' : 0.1,
- Best num_boost_round: 122
- Best CV score: 2.3445089434874253
- Wall time: 12min 2s (local)
- 評価結果  : 2.3562648